In [ ]:
import pandas as pd
import os
import word2number as w2n  # Si lo usas en otros cálculos, de lo contrario se puede omitir
import re
import openpyxl


csv_file = "C:/Users/Carlo/Desktop/SPRINTS/Sprint_10/Formulario_corregido_python.csv"
df = pd.read_csv(csv_file, encoding="utf-8")

# Normalizar nombres de columna: quitar saltos de línea y espacios extra
df.columns = (
    df.columns
      .str.replace(r"\r?\n", " ", regex=True)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)

socio_mapping = {
    "Edad (en años)":              "edad",
    "Género":                      "genero",
    "Código Postal donde reside": "ubicacion",
    "Tipo de área donde reside":  "tipo_area",
    "Nivel de educación alcanzado": "nivel_educacion",
    "Inclinación política":        "inclinacion_politica"
}

df_encuestado = df[list(socio_mapping.keys())].rename(columns=socio_mapping)
df_encuestado.insert(0, "id_encuestado", range(1, len(df_encuestado) + 1))



#Transformaciones

df_encuestado["edad"] = df_encuestado["edad"].replace({"Trentaydos": "32"}).astype(int)

df["En una escala del 1 al 4, donde 1 es 'muy bajo' y 4 es 'muy alto', ¿Cómo calificaría su nivel de estrés relacionado con el trabajo o el estudio?"] = \
    df["En una escala del 1 al 4, donde 1 es 'muy bajo' y 4 es 'muy alto', ¿Cómo calificaría su nivel de estrés relacionado con el trabajo o el estudio?"] \
    .replace({"Muy alto ": 4}).apply(lambda x: float(x) if str(x).isdigit() else 4 if "alto" in str(x).lower() else None)

df["En una escala del 1 al 10, ¿Cómo calificaría su nivel actual de ansiedad?"] = \
    df["En una escala del 1 al 10, ¿Cómo calificaría su nivel actual de ansiedad?"].apply(
        lambda x: float(x.replace(",", ".")) if isinstance(x, str) else x
    )

df["¿Cuántas horas duerme en promedio por noche?"] = df["¿Cuántas horas duerme en promedio por noche?"].apply(
    lambda x: re.sub(r"horas|H|h", "", x) if isinstance(x, str) else x
)

df["Escriba cinco palabras que asocie con la ansiedad"] = (
    df["Escriba cinco palabras que asocie con la ansiedad"]
    .astype(str)  # Asegurar que todos los valores sean strings
    .fillna("")   # Evitar problemas con valores nulos
    .str.replace("[,\\-.\\\\]", "", regex=True)  # Eliminar signos de puntuación
    .str.replace(r"\b(y|e)\b", "", regex=True)  # Eliminar "y" y "e" como palabras individuales
    .str.replace("  ", " ", regex=True)  # Reemplazar dobles espacios por uno solo
    .str.strip()  # Eliminar espacios al inicio y final
)

df["Escriba cinco palabras que asocie con un período sin ansiedad o con momentos de calma"] = (
    df["Escriba cinco palabras que asocie con un período sin ansiedad o con momentos de calma"]
    .astype(str)
    .fillna("")
    .str.replace("[,\\-.\\\\]", "", regex=True)
    .str.replace(r"\b(y|e)\b", "", regex=True)
    .str.replace("  ", " ", regex=True)
    .str.strip()
)

df["¿Cuántas comidas realiza al día por norma general?"] = (
    df["¿Cuántas comidas realiza al día por norma general?"]
    .astype(str)
    .str.lower()
    .replace({"una": 1, "dos": 2, "tres": 3, "cuatro": 4, "cinco": 5, "seis": 6, "siete": 7})
)


#Crear tabla pregunta

preguntas = [
    {"id_pregunta":  1, "texto_pregunta": "Marca temporal", "tipo": "datetime", "seccion": "Metadatos"},
    {"id_pregunta":  2, "texto_pregunta": "Edad (en años)", "tipo": "numérica", "seccion": "Sociodemográficos"},
    {"id_pregunta":  3, "texto_pregunta": "Género", "tipo": "categórica", "seccion": "Sociodemográficos"},
    {"id_pregunta":  4, "texto_pregunta": "Código Postal donde reside", "tipo": "texto", "seccion": "Sociodemográficos"},
    {"id_pregunta":  5, "texto_pregunta": "Tipo de área donde reside", "tipo": "categórica", "seccion": "Sociodemográficos"},
    {"id_pregunta":  6, "texto_pregunta": "Nivel de educación alcanzado", "tipo": "categórica", "seccion": "Sociodemográficos"},
    {"id_pregunta":  7, "texto_pregunta": "Inclinación política", "tipo": "categórica", "seccion": "Sociodemográficos"},
    {"id_pregunta":  8, "texto_pregunta": "¿Alguna vez le han diagnosticado algún trastorno de ansiedad?", "tipo": "categórica", "seccion": "Salud y Ansiedad"},
    {"id_pregunta":  9, "texto_pregunta": "En una escala del 1 al 10, ¿Cómo calificaría su nivel actual de ansiedad?", "tipo": "numérica", "seccion": "Salud y Ansiedad"},
    {"id_pregunta": 10, "texto_pregunta": "Durante las últimas dos semanas, ¿con qué frecuencia ha experimentado preocupación excesiva? (0 = Ningún día, 1 = Varios días, 2 = Más de la mitad de los días, 3 = Casi todos los días)", "tipo": "ordinal", "seccion": "Salud y Ansiedad"},
    {"id_pregunta": 11, "texto_preggunta": "Durante las últimas dos semanas, ¿con qué frecuencia ha tenido dificultad para relajarse? (0 = Ningún día, 1 = Varios días, 2 = Más de la mitad de los días, 3 = Casi todos los días)", "tipo": "ordinal", "seccion": "Salud y Ansiedad"},
    {"id_pregunta": 12, "texto_pregunta": "Durante las últimas dos semanas, ¿con qué frecuencia ha sentido nerviosismo o estrés? (0 = Ningún día, 1 = Varios días, 2 = Más de la mitad de los días, 3 = Casi todos los días)", "tipo": "ordinal", "seccion": "Salud y Ansiedad"},
    {"id_pregunta": 13, "texto_pregunta": "¿Ha sufrido un ataque de ansiedad o pánico en los últimos meses?", "tipo": "categórica", "seccion": "Salud y Ansiedad"},
    {"id_pregunta": 14, "texto_pregunta": "En caso afirmativo, ¿hace cuánto tiempo ocurrió el último ataque de ansiedad o pánico? (indique el tiempo en semanas)", "tipo": "numérica", "seccion": "Salud y Ansiedad"},
    {"id_pregunta": 15, "texto_pregunta": "Durante la última semana, ¿ha practicado alguna actividad física o deporte?", "tipo": "categórica", "seccion": "Estilo de Vida"},
    {"id_pregunta": 16, "texto_pregunta": "Si respondió 'Sí', ¿Cuántas veces practicó actividad física o deporte en la última semana?", "tipo": "numérica", "seccion": "Estilo de Vida"},
    {"id_pregunta": 17, "texto_pregunta": "En promedio, ¿Cuántas horas diarias permanece sentado/a en actividades de trabajo?", "tipo": "numérica", "seccion": "Estilo de Vida"},
    {"id_pregunta": 18, "texto_pregunta": "En promedio, ¿Cuántas horas diarias permanece sentado/a en actividades de ocio o tiempo personal?", "tipo": "numérica", "seccion": "Estilo de Vida"},
    {"id_pregunta": 19, "texto_pregunta": "En promedio, ¿Cuántas horas diarias permanece sentado/a en transporte?", "tipo": "numérica", "seccion": "Estilo de Vida"},
    {"id_pregunta": 20, "texto_pregunta": "¿Con qué frecuencia presenta dificultades para conciliar el sueño?", "tipo": "ordinal", "seccion": "Estilo de Vida"},
    {"id_pregunta": 21, "texto_pregunta": "¿Cuántas horas duerme en promedio por noche?", "tipo": "numérica", "seccion": "Estilo de Vida"},
    {"id_pregunta": 22, "texto_pregunta": "¿Cómo describiría su dieta? (Puede seleccionar varias opciones)", "tipo": "categórica", "seccion": "Alimentación y Dieta"},
    {"id_pregunta": 23, "texto_pregunta": "¿Cuántas comidas realiza al día por norma general?", "tipo": "numérica", "seccion": "Alimentación y Dieta"},
    {"id_pregunta": 24, "texto_pregunta": "En promedio, ¿Cuántas porciones de frutas y verduras ha consumido diariamente la última semana?", "tipo": "numérica", "seccion": "Alimentación y Dieta"},
    {"id_pregunta": 25, "texto_pregunta": "¿Cuántas tazas de café (u otras bebidas con cafeína) consume al día?", "tipo": "numérica", "seccion": "Alimentación y Dieta"},
    {"id_pregunta": 26, "texto_pregunta": "¿Cuántas porciones de alimentos ultraprocesados consume en un día típico?", "tipo": "numérica", "seccion": "Alimentación y Dieta"},
    {"id_pregunta": 27, "texto_preggunta": "¿Con qué frecuencia siente que consume alimentos debido a ansiedad o estrés?", "tipo": "categórica", "seccion": "Alimentación y Dieta"},
    {"id_pregunta": 28, "texto_preggunta": "Cuando experimenta ansiedad, ¿cómo suele afectar su comportamiento alimenticio?", "tipo": "categórica", "seccion": "Alimentación y Dieta"},
    {"id_pregunta": 29, "texto_pregunta": "En una escala del 1 al 4, donde 1 es 'muy bajo' y 4 es 'muy alto', ¿Cómo calificaría su nivel de estrés relacionado con el trabajo o el estudio?", "tipo": "numérica", "seccion": "Factores Contextuales"},
    {"id_pregunta": 30, "texto_pregunta": "¿Considera que tiene un buen balance entre su vida laboral/estudio y personal?", "tipo": "categórica", "seccion": "Factores Contextuales"},
    {"id_pregunta": 31, "texto_preggunta": "Aproximadamente, ¿Cuántas horas diarias dedica a redes sociales o a actividades en dispositivos electrónicos?", "tipo": "numérica", "seccion": "Factores Contextuales"},
    {"id_pregunta": 32, "texto_preggunta": "Escriba cinco palabras que asocie con la ansiedad", "tipo": "texto", "seccion": "Preguntas Abiertas"},
    {"id_pregunta": 33, "texto_preggunta": "Escriba cinco palabras que asocie con un período sin ansiedad o con momentos de calma", "tipo": "texto", "seccion": "Preguntas Abiertas"}
]

df_pregunta = pd.DataFrame(preguntas)

#Transformar preguntas a formato relacional 

#Construir un diccionario de mapeo para asociar cada texto de pregunta
columna_a_id = {
    (p.get("texto_pregunta") or p.get("texto_preggunta")).strip(): p["id_pregunta"]
    for p in preguntas
}

#Añadir el id_encuestado al DataFrame original 
df_temp = df.copy()
df_temp["id_encuestado"] = df_encuestado["id_encuestado"]

# 4.3. Convertir el DataFrame de respuestas (formato ancho) a formato largo
df_respuestas = pd.melt(
    df_temp,
    id_vars=["id_encuestado"],
    var_name="pregunta_texto",
    value_name="respuesta"
)
df_respuestas["pregunta_texto"] = df_respuestas["pregunta_texto"].str.strip()

#Mapear el texto de la pregunta a su id_pregunta usando el diccionario
df_respuestas["id_pregunta"] = df_respuestas["pregunta_texto"].map(columna_a_id)
faltantes = df_respuestas.loc[df_respuestas["id_pregunta"].isna(), "pregunta_texto"].unique()
if len(faltantes) > 0:
    print("Preguntas sin ID asignado:", faltantes)
df_respuestas["id_pregunta"] = df_respuestas["id_pregunta"].fillna(-1).astype(int)

df_respuestas = df_respuestas.merge(
    df_pregunta[["id_pregunta", "seccion", "texto_pregunta"]],
    on="id_pregunta",
    how="left"
)

#Crear Tabla pivot

# Crear una vista pivot donde cada columna corresponde a una pregunta y contiene la respuesta de cada encuestado.
df_pivot = df_respuestas.pivot_table(
    index="id_encuestado",  
    columns="id_pregunta",
    values="respuesta",
    aggfunc="first",
    fill_value="Sin datos"
).reset_index()  # 🔹 Convertir el índice en columna

# Renombrar las columnas del pivot para que muestren el texto completo de la pregunta
map_id_to_text = df_pregunta.set_index("id_pregunta")["texto_pregunta"].to_dict()
df_pivot.rename(columns=map_id_to_text, inplace=True)


#Exportar tablas

output_dir = os.getcwd()  
print(f"Los archivos se han guardado en: {output_dir}")

# Exportar tablas relacionales
df_encuestado.to_csv("Encuestado.csv", index=False)
df_pregunta.to_csv("Pregunta.csv", index=False)
df_respuestas.to_csv("Respuesta_completa.csv", index=False)
df_pivot.to_csv("pivot_table.csv", index=False)

In [ ]:
# 1. Cargar el CSV y normalizar encabezados
csv_file = "C:/Users/Carlo/Desktop/SPRINTS/Sprint_10/pivot_table_origen.csv"
df = pd.read_csv(csv_file, encoding="utf-8")


df_pivot = df_pivot.astype({"Si respondió 'Sí', ¿Cuántas veces practicó actividad física o deporte en la última semana?": int})

df_pivot = df_pivot.astype({"En una escala del 1 al 10, ¿Cómo calificaría su nivel actual de ansiedad?": int})

df_pivot = df_pivot.astype({"En una escala del 1 al 4, donde 1 es 'muy bajo' y 4 es 'muy alto', ¿Cómo calificaría su nivel de estrés relacionado con el trabajo o el estudio?": int})


df_pivot["En promedio, ¿Cuántas porciones de frutas y verduras ha consumido diariamente la última semana?"] = (
    pd.to_numeric(
        df_pivot["Si respondió 'Sí', ¿Cuántas veces practicó actividad física o deporte en la última semana?"],
        errors="coerce"  # 🔹 Convierte números y deja texto como NaN
    )
    .astype("Int64")  # 🔹 Convertir a número entero, manteniendo NaN
)
df_pivot.to_csv("pivot_table_final.csv", index=False)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Convertir columnas categóricas a numéricas
df_pivot["estres_laboral"] = pd.to_numeric(df_pivot["En una escala del 1 al 4, donde 1 es 'muy bajo' y 4 es 'muy alto', ¿Cómo calificaría su nivel de estrés relacionado con el trabajo o el estudio?"], errors="coerce")
df_pivot["preocupacion_excesiva"] = pd.to_numeric(
    df_pivot["Durante las últimas dos semanas, ¿con qué frecuencia ha experimentado preocupación excesiva? (0 = Ningún día, 1 = Varios días, 2 = Más de la mitad de los días, 3 = Casi todos los días)"],
    errors="coerce"
)
df_pivot["nerviosismo_estres"] = pd.to_numeric(df_pivot["Durante las últimas dos semanas, ¿con qué frecuencia ha sentido nerviosismo o estrés? (0 = Ningún día, 1 = Varios días, 2 = Más de la mitad de los días, 3 = Casi todos los días)"
                                                              ], errors="coerce")
df_pivot["dificultad_relajarse"] = pd.to_numeric(df_pivot[
    "Durante las últimas dos semanas, ¿con qué frecuencia ha tenido dificultad para relajarse? (0 = Ningún día, 1 = Varios días, 2 = Más de la mitad de los días, 3 = Casi todos los días)"], errors="coerce")

df_clean = df_pivot.dropna(subset=["estres_laboral", "preocupacion_excesiva", "nerviosismo_estres", "dificultad_relajarse"])

correlacion = df_clean[["estres_laboral", "preocupacion_excesiva", "nerviosismo_estres", "dificultad_relajarse"]].corr()

print("Matriz de correlación:")
print(correlacion)

plt.figure(figsize=(8, 6))
sns.heatmap(correlacion, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación entre Estrés, Preocupación y Nerviosismo")
plt.show()

In [ ]:
#Finalmente no usado
conteo_manual = df_encuestado["ubicacion"].astype(str).str[:2].value_counts().to_dict()

provincias_españa = {
    '01': "Álava", '02': "Albacete", '03': "Alicante", '04': "Almería", '05': "Ávila",
    '06': "Badajoz", '07': "Islas Baleares", '08': "Barcelona", '09': "Burgos", '10': "Cáceres",
    '11': "Cádiz", '12': "Castellón", '13': "Ciudad Real", '14': "Córdoba", '15': "A Coruña",
    '16': "Cuenca", '17': "Girona", '18': "Granada", '19': "Guadalajara", '20': "Gipuzkoa",
    '21': "Huelva", '22': "Huesca", '23': "Jaén", '24': "León", '25': "Lleida",
    '26': "La Rioja", '27': "Lugo", '28': "Madrid", '29': "Málaga", '30': "Murcia",
    '31': "Navarra", '32': "Ourense", '33': "Asturias", '34': "Palencia", '35': "Las Palmas",
    '36': "Pontevedra", '37': "Salamanca", '38': "Santa Cruz de Tenerife", '39': "Cantabria", '40': "Segovia",
    '41': "Sevilla", '42': "Soria", '43': "Tarragona", '44': "Teruel", '45': "Toledo",
    '46': "Valencia", '47': "Valladolid", '48': "Bizkaia", '49': "Zamora", '50': "Zaragoza",
    '51': "Ceuta", '52': "Melilla"
}

df_encuestado["provincia"] = df_encuestado["ubicacion"].astype(str).str[:2].map(provincias_españa)
# Convertir el diccionario en un DataFrame
df_provincias = pd.DataFrame(list(provincias_españa.items()), columns=["codigo_postal", "provincia"])

df_encuestado["ubicacion"] = df_encuestado["ubicacion"].str.strip()
df_provincias["codigo_postal"] = df_provincias["codigo_postal"].str.strip()

codigos_faltantes = set(df_encuestado["ubicacion"]) - set(df_provincias["codigo_postal"])

df_encuestado["codigo_postal_2digitos"] = df_encuestado["ubicacion"].astype(str).str[:2]


df_encuestado = pd.merge(df_encuestado, df_provincias, left_on="codigo_postal_2digitos", right_on="codigo_postal", how="left")

df_encuestado.drop(columns=["codigo_postal_2digitos", "codigo_postal", "provincia_y"], inplace=True)

df_pivot.to_csv("pivot_table_final.csv", index=False)

NameError: name 'df_encuestado' is not defined